In [ ]:
import os
import sys
import pandas as pd
sys.path.append(os.path.abspath(os.path.join('..', 'src', 'utils')))
import utils as ut
import ast

# Entendimiento general de datos

In [130]:
wines_df = pd.read_csv("../src/data/processed/all_wines_complete.csv")
wines_df.head()

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,alcohol,image
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020,Luigi Bosca,4.8,582,188.33,73.43435%,50.905894999999994%,13.619732499999998%,44.74824%,"{'black fruit': 14, 'oaky': 13, 'earthy': 5, '...","['beef', 'lamb', 'poultry', 'game (deer, venis...","['Cabernet Sauvignon', 'Malbec']","['Argentina', 'Mendoza']",Argentinian Cabernet Sauvignon - Malbec,NaN,https://images.vivino.com/thumbs/_Bf6JTwYRpSX6...
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015,Catena Zapata,4.7,297,675.0,74.17103499999999%,55.8328075%,14.345029999999998%,54.45488%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,14%,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017,Catena Zapata,4.7,219,580.0,74.17103499999999%,55.8328075%,14.345029999999998%,54.45488%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,NaN,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
3,https://www.vivino.com/US/en/monteviejo-la-vio...,La Violeta,2013,Monteviejo,4.7,202,150.0,73.06289000000001%,40.9165975%,21.4814425%,36.61685%,"{'oaky': 51, 'black fruit': 49, 'earthy': 26, ...","['beef', 'lamb']",['100%Malbec'],"['Argentina', 'Mendoza', 'Uco Valley']",Argentinian Uco Valley Malbec Red,14.5%,https://images.vivino.com/thumbs/FO-x9h3mQHSx9...
4,https://www.vivino.com/US/en/vina-cobos-cobos-...,Cobos Volturno,2018,Viña Cobos,4.7,198,399.99,71.052235%,45.63599%,24.262824999999996%,38.4480575%,"{'oaky': 86, 'black fruit': 49, 'spices': 17, ...","['beef', 'lamb', 'poultry', 'game (deer, venis...",['100%Cabernet Sauvignon'],"['Argentina', 'Mendoza', 'Perdriel']",Argentinian Cabernet Sauvignon - Malbec,NaN,https://images.vivino.com/thumbs/D1Mf1fYnRnage...


In [131]:
wines_df.shape

(2026, 18)

In [132]:
wines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   wine_link   2026 non-null   object 
 1   name        2026 non-null   object 
 2   year        2023 non-null   object 
 3   winery      2026 non-null   object 
 4   rating      2026 non-null   float64
 5   rating_qty  2026 non-null   object 
 6   price       2009 non-null   object 
 7   body        1974 non-null   object 
 8   tannis      1663 non-null   object 
 9   sweetness   1945 non-null   object 
 10  acidity     1974 non-null   object 
 11  notes       2026 non-null   object 
 12  pairings    1974 non-null   object 
 13  grapes      2012 non-null   object 
 14  region      2026 non-null   object 
 15  style       1974 non-null   object 
 16  alcohol     979 non-null    object 
 17  image       2026 non-null   object 
dtypes: float64(1), object(17)
memory usage: 285.0+ KB


# Transformación de datos

## Pasos:

1. **`Tipo de Dato Float`**: convierte las columnas **`[year, rating_qty, price]`** a numéricas.
- **`year`**: existen vinos sin año que traen el dato "N.V.". Los pasamos a NaN.
- **`rating_qty`**: cuando los vinos no tienen suficientes ratings, los basan en los vinos de otros años. En este caso, los pasamos a NaN.

2. **`Tastes + Alcohol`**: convierte las columnas **`[body, tannis, sweetness, acidity, alcohol]`** en decimales.

3. **`Diccionarios`**: transforma la columna **`{notes}`** a formato tipo *one-hot* normalizado, donde la suma de todas las notas de un vino da 1.

4. **`Listas`**: splitea y transforma las columnas **`[pairings, grapes, region]`** a formato *one-hot*.

In [143]:
# Entendimiento de valores nulos year, rating_qyt, price
col = "price" 
mask_invalid = pd.to_numeric(wines_df[col], errors="coerce").isna() & wines_df[col].notna()
wines_df[mask_invalid]

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,alcohol,image
1635,https://www.vivino.com/catena-appellation-en-f...,Appellation En Fuego Malbec,2013,Catena,4.1,based,"1,438.99",59.347825%,21.85055%,15.054347499999999%,23.4782625%,{},"['beef', 'lamb', 'pork']",['Malbec'],"['Argentina', 'Mendoza']",Argentinian Mendoza Malbec Red,NaN,https://images.vivino.com/thumbs/TwuvDFXuToi7l...


In [144]:
# Year, Rating Qty, Price
winesdf_dtypes = wines_df.copy()
columns_to_convert = ["year", "rating_qty", "price"]
for col in columns_to_convert:
    winesdf_dtypes[col] = pd.to_numeric(winesdf_dtypes[col].str.replace(",", ""), errors="coerce")
winesdf_dtypes["year"] = winesdf_dtypes["year"].astype("Int64")
winesdf_dtypes.info()
winesdf_dtypes.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   wine_link   2026 non-null   object 
 1   name        2026 non-null   object 
 2   year        1998 non-null   Int64  
 3   winery      2026 non-null   object 
 4   rating      2026 non-null   float64
 5   rating_qty  1606 non-null   float64
 6   price       2009 non-null   float64
 7   body        1974 non-null   object 
 8   tannis      1663 non-null   object 
 9   sweetness   1945 non-null   object 
 10  acidity     1974 non-null   object 
 11  notes       2026 non-null   object 
 12  pairings    1974 non-null   object 
 13  grapes      2012 non-null   object 
 14  region      2026 non-null   object 
 15  style       1974 non-null   object 
 16  alcohol     979 non-null    object 
 17  image       2026 non-null   object 
dtypes: Int64(1), float64(3), object(14)
memory usage: 287.0+ KB


,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,alcohol,image
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020,Luigi Bosca,4.8,582.0,188.33,73.43435%,50.905894999999994%,13.619732499999998%,44.74824%,"{'black fruit': 14, 'oaky': 13, 'earthy': 5, '...","['beef', 'lamb', 'poultry', 'game (deer, venis...","['Cabernet Sauvignon', 'Malbec']","['Argentina', 'Mendoza']",Argentinian Cabernet Sauvignon - Malbec,NaN,https://images.vivino.com/thumbs/_Bf6JTwYRpSX6...
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015,Catena Zapata,4.7,297.0,675.00,74.17103499999999%,55.8328075%,14.345029999999998%,54.45488%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,14%,https://images.vivino.com/thumbs/Yt464jw0QS-ug...
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017,Catena Zapata,4.7,219.0,580.00,74.17103499999999%,55.8328075%,14.345029999999998%,54.45488%,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,NaN,https://images.vivino.com/thumbs/Yt464jw0QS-ug...


In [145]:
# Tastes
def clean_and_convert_tastes(value):
    if isinstance(value, float):  
        return round(value/100,4)
    return round(pd.to_numeric(str(value)[:5].replace('%', ''), errors='coerce')/100,4)

winesdf_taste = winesdf_dtypes.copy()

columns_to_convert = ['body', 'tannis', 'sweetness', 'acidity', 'alcohol']
for col in columns_to_convert:
    winesdf_taste[col] = winesdf_taste[col].apply(clean_and_convert_tastes)

winesdf_taste.info()
winesdf_taste.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   wine_link   2026 non-null   object 
 1   name        2026 non-null   object 
 2   year        1998 non-null   Int64  
 3   winery      2026 non-null   object 
 4   rating      2026 non-null   float64
 5   rating_qty  1606 non-null   float64
 6   price       2009 non-null   float64
 7   body        1974 non-null   float64
 8   tannis      1663 non-null   float64
 9   sweetness   1945 non-null   float64
 10  acidity     1974 non-null   float64
 11  notes       2026 non-null   object 
 12  pairings    1974 non-null   object 
 13  grapes      2012 non-null   object 
 14  region      2026 non-null   object 
 15  style       1974 non-null   object 
 16  alcohol     979 non-null    float64
 17  image       2026 non-null   object 
dtypes: Int64(1), float64(8), object(9)
memory usage: 287.0+ KB


,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,acidity,notes,pairings,grapes,region,style,alcohol,image
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020,Luigi Bosca,4.8,582.0,188.33,0.7343,0.5090,0.1361,0.4474,"{'black fruit': 14, 'oaky': 13, 'earthy': 5, '...","['beef', 'lamb', 'poultry', 'game (deer, venis...","['Cabernet Sauvignon', 'Malbec']","['Argentina', 'Mendoza']",Argentinian Cabernet Sauvignon - Malbec,NaN,https://images.vivino.com/thumbs/_Bf6JTwYRpSX6...
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015,Catena Zapata,4.7,297.0,675.00,0.7417,0.5583,0.1434,0.5445,"{'oaky': 109, 'black fruit': 50, 'spices': 25,...","['beef', 'lamb', 'poultry', 'pasta']","['Cabernet Sauvignon', 'Malbec', 'Merlot']","['Argentina', 'Mendoza', 'Agrelo']",Argentinian Bordeaux Blend,0.14,https://images.vivino.com/thumbs/Yt464jw0QS-ug...


In [146]:
# Funciones para gestionar tipos de datos complejos en columnas (listas y diccionarios)
# ast.literal_eval : https://medium.com/@aniruddhapal/the-power-of-the-ast-literal-eval-method-in-python-8fb4014a2574

def extract_unique_values(df_col, type="list"):
    """Procesa el diccionario almacenado como string y crea un set de las palabras que contiene."""
    unique_values = set()
    for entry in df_col.dropna():
        try:
            values = ast.literal_eval(entry) # Convierte string en list o dictionary
            if type == "list":
                unique_values.update(values) # Agrega calores de la list al set()
            elif type == "dict":
                unique_values.update(values.keys()) # Agrega keys del dict al set()
            else:
                raise TypeError("Only types supported are 'list' and 'dict'!")
        except (SyntaxError, ValueError):
            # Incorporar logging con error
            continue
    return sorted(unique_values)

def split_list_cols(df, col):
    unique_values = extract_unique_values(df[col], type="list")
    splitted_df = df.copy()
    for val in unique_values:
        splitted_df[val] = splitted_df[col].apply(
            lambda x: None if pd.isna(x) else 1 if val in ast.literal_eval(x) else 0
        ).astype("Int64")
    return splitted_df.drop(columns=[col])

In [147]:
# Notes
def split_normalize_notes(df):
    unique_notes = extract_unique_values(df["notes"], type="dict")
    
    splitted_df = df.copy()

    for note in unique_notes:
        splitted_df[note] = splitted_df["notes"].apply(
            lambda x: ast.literal_eval(x).get(note, 0) if pd.notna(x) else None
        )

    note_sums = splitted_df[unique_notes].sum(axis=1)
    splitted_df[unique_notes] = splitted_df[unique_notes].div(note_sums, axis=0).fillna(0).round(4) # Normaliza
    return splitted_df.drop(columns=["notes"])

# Aplicar la función al DataFrame
winesdf_note = split_normalize_notes(winesdf_taste)
winesdf_note.head(3)

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,...,dried fruit,earthy,floral,oaky,red fruit,spices,tree fruit,tropical,vegetal,yeasty
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020,Luigi Bosca,4.8,582.0,188.33,0.7343,0.5090,0.1361,...,0.000,0.1250,0.0500,0.3250,0.0500,0.1000,0.0,0.0,0.000,0.000
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015,Catena Zapata,4.7,297.0,675.00,0.7417,0.5583,0.1434,...,0.012,0.0964,0.0241,0.4378,0.0843,0.1004,0.0,0.0,0.004,0.008
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017,Catena Zapata,4.7,219.0,580.00,0.7417,0.5583,0.1434,...,0.012,0.0964,0.0241,0.4378,0.0843,0.1004,0.0,0.0,0.004,0.008


In [148]:
# Pairings, Grapes, Regions
list_columns = ["pairings", "grapes", "region"]

winesdf_note["grapes"] = winesdf_note["grapes"].replace(r"\d+%", "", regex=True)

winedf_tra = winesdf_note.copy()

for col in list_columns:
    winedf_tra = split_list_cols(winedf_tra, col)

# winedf_tra[extract_unique_values(winesdf_note["region"], "list")].head() # Revisar valores de una list col

winedf_tra.head(3)

,wine_link,name,year,winery,rating,rating_qty,price,body,tannis,sweetness,...,San Carlos,San Juan,San Rafael,Serra Gaúcha,Tulum Valley,Tunuyán,Tupungato,Uco Valley,Vale dos Vinhedos,Vista Flores
0,https://www.vivino.com/US/en/luigi-bosca-parai...,Paraiso,2020,Luigi Bosca,4.8,582.0,188.33,0.7343,0.5090,0.1361,...,0,0,0,0,0,0,0,0,0,0
1,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2015,Catena Zapata,4.7,297.0,675.00,0.7417,0.5583,0.1434,...,0,0,0,0,0,0,0,0,0,0
2,https://www.vivino.com/US/en/catena-zapata-est...,Estiba Reservada,2017,Catena Zapata,4.7,219.0,580.00,0.7417,0.5583,0.1434,...,0,0,0,0,0,0,0,0,0,0


In [149]:
winedf_tra.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2026 entries, 0 to 2025
Columns: 119 entries, wine_link to Vista Flores
dtypes: Int64(93), float64(21), object(5)
memory usage: 2.0+ MB


In [150]:
# Guardado final del dataset transformado
ut.save_csv(winedf_tra, path="../src/data/transformed/", filename="wines_transformed.csv")

Archivo guardado en: ../src/data/transformed/wines_transformed.csv
